<a href="https://colab.research.google.com/github/mathewpolonsky/Marketplace-Item-Matching/blob/main/Training_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1xqbHyg30TSoXUnVALYkuLlgHfFUPFyM6
!gdown 1AgER8HsTYSKmdY2q40qdoCsiCDp7SvIT
!gdown 1FwNz9suT_aEYCHstliz8YmRz2uTfqwIe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

---

In [ ]:
import json
from datetime import datetime

import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## Training CatBoost

In [ ]:
basic_cols = ['target', 'variantid1', 'variantid2']

train = pd.merge(    
    pd.read_csv("train_cb_rubert_base_class_full_2_eps.csv"),
    pd.read_csv("train_cb_xlm_roberta_base_regr_diff_2_eps.csv"),
    on=basic_cols
).merge(pd.read_parquet("merged_train.parquet"), on=basic_cols).merge(
    pd.read_csv("train_cb_distilbert_quora_full_6_eps.csv"), on=basic_cols)

train

In [ ]:
train = train.drop(['full_text_1', 'full_text_2'], axis=1)
train.head(2)

In [ ]:
train["main_pic_embed_cos_sim"] = train.apply(lambda x: cosine_similarity([x['main_pic_embeddings_resnet_v1_1'][0]],
                                                                          [x['main_pic_embeddings_resnet_v1_2'][0]])[0][0], axis=1)

train = train.drop(['main_pic_embeddings_resnet_v1_1', 'main_pic_embeddings_resnet_v1_2'], axis=1)

train.head(2)

In [ ]:
X_train, X_val = train_test_split(train, test_size=0.1, random_state=42)

y_train = X_train["target"]
y_val = X_val["target"]

X_train = X_train.drop(["target"], axis=1)
X_val = X_val.drop(["target"], axis=1)

train_pool = Pool(
    data=X_train[train.columns[3:]],
    label=y_train
)
eval_pool = Pool(
    data=X_val[train.columns[3:]],
    label=y_val
)

In [ ]:
model = CatBoostClassifier(
    task_type='GPU',
    iterations=1000,
    random_seed=63,
    learning_rate=0.01,
    depth=10,
    l2_leaf_reg=3
)

In [ ]:
model.fit(
    train_pool,
    eval_set=eval_pool,
    plot=True,
    verbose=True,
    use_best_model=True,
    early_stopping_rounds=50,
    metric_period=100,
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6814309	test: 0.6826879	best: 0.6826879 (0)	total: 121ms	remaining: 2m
100:	learn: 0.3072237	test: 0.3711034	best: 0.3711034 (100)	total: 7.7s	remaining: 1m 8s
200:	learn: 0.2768718	test: 0.3625701	best: 0.3614121 (169)	total: 14.7s	remaining: 58.3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3614120542
bestIteration = 169

Shrink model to first 170 iterations.


In [ ]:
nlp_models_names = 'xlm_roberta_regr_diff_2_eps+rubert_base_class_full_2_eps+distilbert_quora_full_6_eps+main_pics_cos_sim'

In [ ]:
save_path = f"drive/MyDrive/models/catboost+{nlp_models_names}.cbm"

model.save_model(save_path)
print(save_path)

drive/MyDrive/models/catboost+xlm_roberta_regr_diff_2_eps+rubert_base_class_full_2_eps+distilbert_quora_full_6_eps+main_pics_cos_sim.cbm
